In [6]:
import numpy as np
import math
import time
import pickle
from scipy import optimize
import polyscope as ps
from barmesh.tribarmes import trianglebarmesh
from barmesh.basicgeo import P3
import barmesh.geodesicUtils as geo

In [7]:
fname = 'geometry/flat_square.stl'

tbm = trianglebarmesh.TriangleBarMesh(fname)


V = tbm.GetVertices()
F = tbm.GetFaces()
n = tbm.GetNormals()

Triangle bar mesh loaded with 1036 triangles of average size 4.8


In [21]:
startFace = 898
x,y,z = 52.5,0,0
#for n in tbm.faces[startFace].nodes:
#    x += n.p.x/3
#    y += n.p.y/3
#    z += n.p.z/3
startPt = P3(x,y,z)

theta = 0.01
ref0 = P3(0,1,0)
geoline = createGeoLine(tbm, startPt, startFace, theta, ref0, calc_thick = False, maxPathLength = 10000)
print(geoline['length'])

edge reached on path 0.01 deg
100.00000152308718


In [22]:
ps.init()

ps_surf = ps.register_surface_mesh("my mesh", V, F, edge_width = 1, smooth_shade=False)
ps_geoline = ps.register_curve_network('geoline', geo.P3list2array(geoline['pts']), 'line')

ps.show()

In [20]:
#function to rotate a P3 vector about an axis (of any length) and an angle in degrees
def rotAxisAngle(v,ax,a):
    ax = P3.ZNorm(ax)
    c = np.cos(np.radians(a))
    s = np.sin(np.radians(a))
    C = 1 - np.cos(np.radians(a))
    Q= np.array(((ax.x*ax.x*C+c,      ax.x*ax.y*C-ax.z*s, ax.x*ax.z*C+ax.y*s),
                 (ax.y*ax.x*C+ax.z*s, ax.y*ax.y*C+c,      ax.y*ax.z*C-ax.x*s),
                 (ax.z*ax.x*C-ax.y*s, ax.z*ax.y*C+ax.x*s, ax.z*ax.z*C+c)))
    v = np.dot(Q,np.array((v.x,v.y,v.z)))
    return P3(v[0],v[1],v[2])

def Square(X):
    return X*X
def TOL_ZERO(X):
    if not (abs(X) < 0.0001):
        print("TOL_ZERO fail", X)

def GeoCrossAxisE(a, Vae, Vab, Isq, Isgn):
    # Solve: Isq*x.Lensq() - Square(P3.Dot(x, Vab)) = 0   for x = a + Vae*q
    # 0 = Isq*(a^2 + 2q a.Vae + q^2 Vae^2) - (a.Vab + Vae.Vab q)^2
    #   = Isq*(a^2 + 2q adf + q^2 Vae^2) - (adv + fdv q)^2

    fdv = P3.Dot(Vae, Vab)
    adv = P3.Dot(a, Vab)
    adf = P3.Dot(a, Vae)
    qA = Square(fdv) - Vae.Lensq()*Isq
    qB2 = adv*fdv - adf*Isq
    qC = Square(adv) - a.Lensq()*Isq
    if abs(qA) <= abs(qB2)*1e-7:
        if qB2 == 0:
            return -1.0
        q = -qC/(2*qB2)
    else:
        qdq = Square(qB2) - qA*qC
        if qdq < 0.0:
            #print("qdq", qdq)
            return -1.0
        qs = math.sqrt(qdq) / qA
        qm = -qB2 / qA
        q = qm + qs*Isgn
    # q = qs +- qm,  x = a + Vae*q,  Dot(x, Vab) same sign as Dot(Vcd, Vab)
    if abs(q) < 100:
        TOL_ZERO(qA*Square(q) + qB2*2*q + qC)
    return q

#
# This is the basic function that crosses from one triangle to the next
#
def GeoCrossAxis(Ga, Gb, Gc, lam, Ge):
    Vab = Gb - Ga
    Gd = Ga + Vab*lam
    Vcd = Gd - Gc
    if Vcd.Len() == 0:
        bEnd = True
        bAEcrossing = False
        q = 0
        Gx = Gc
    else:
        bEnd = False
        cdDab = P3.Dot(Vcd, Vab)
        Isq = Square(cdDab) / Vcd.Lensq()
        Isgn = -1 if cdDab < 0 else 1
        qVae = GeoCrossAxisE(Ga - Gd, Ge - Ga, Vab, Isq, Isgn)
        qVbe = GeoCrossAxisE(Gb - Gd, Ge - Gb, -Vab, Isq, -Isgn)
        bAEcrossing = (abs(qVae - 0.5) < abs(qVbe - 0.5))
        q = qVae if bAEcrossing else qVbe
        Gx = (Ga + (Ge - Ga)*q) if bAEcrossing else (Gb + (Ge - Gb)*q)
        Dx = Gx - Gd
        TOL_ZERO(Isq - Square(P3.Dot(Dx, Vab)/Dx.Len()))
        TOL_ZERO(P3.Dot(Vcd, Vab)/Vcd.Len() - P3.Dot(Dx, Vab)/Dx.Len())
    return bAEcrossing, q, Gx, bEnd

# This is the iterative function that goes from bar to bar through the mesh
# c=from point, bar the crossing edge, lam the point on the bar, and bGoRight is the crossing direction
#
def GeoCrossBar(c, bar, lam, bGoRight, bPrintvals=False):
    bEnd = False
    Na, Nb = bar.nodeback, bar.nodefore
    if bGoRight:
        if bar.barforeright == None:
            bEnd = True
            Ne = trianglebarmesh.TriangleNode(P3(bar.nodefore.p.x+0.01,bar.nodefore.p.y+0.01,bar.nodefore.p.z+0.01),-1)
        else:
            Ne = bar.barforeright.GetNodeFore(bar.barforeright.nodeback == bar.nodefore)
    else:
        if bar.barbackleft == None:
            bEnd = True
            Ne = trianglebarmesh.TriangleNode(P3(bar.nodeback.p.x+0.01,bar.nodeback.p.y+0.01,bar.nodeback.p.z+0.01),-1)
        else:
            Ne = bar.barbackleft.GetNodeFore(bar.barbackleft.nodeback == bar.nodeback)
    d = Na.p + (Nb.p - Na.p)*lam
    if bPrintvals:
        print("\n", (Na.p, Nb.p, c, lam, Ne.p))
    if not bEnd:
        bAEcrossing, q, Gx, bEnd = GeoCrossAxis(Na.p, Nb.p, c, lam, Ne.p)
    if not bEnd:
        if bGoRight:
            if bAEcrossing:
                bar = bar.barforeright.GetForeRightBL(bar.barforeright.nodeback == Nb)
                lam = q if bar.nodeback == Na else 1-q
                bGoRight = (bar.nodeback == Na)   
            else:
                bar = bar.barforeright
                lam = q if bar.nodeback == Nb else 1-q
                bGoRight = not (bar.nodeback == Nb)
        else:
            if bAEcrossing:
                bar = bar.barbackleft
                lam = q if bar.nodeback == Na else 1-q
                bGoRight = not (bar.nodeback == Na)
            else:
                bar = bar.barbackleft.GetForeRightBL(bar.barbackleft.nodeback == Na)
                lam = q if bar.nodeback == Nb else 1-q
                bGoRight = (bar.nodeback == Nb)
                
        c = bar.nodeback.p + (bar.nodefore.p - bar.nodeback.p)*lam
        TOL_ZERO((c - Gx).Len())
    else:
        bar = None
        c = None
    if bPrintvals:
        print("d,c", (d, c))
    
    return (d, bar, lam, bGoRight)


def clamp(num, min_value, max_value):
        if num < min_value or num > max_value: clamped = True
        else: clamped = False
        num = max(min(num, max_value), min_value)
        return num, clamped

#Function to calculate the shortest distance between point P and line AB
def distPointLine(A,B,P):
    if A != B:
        v = B-A
        x = P3.Dot((P-A), v) / P3.Dot(v, v) #Ratio of distance along line AB of closest point to P
    else:
        v = A
        x = -1
    x, clamped = clamp(x, 0, 1) #Fix point of closest approach to be between the two points
    #print('Point of closest approach:',(A + v*x))
    dist = (A + v*x - P).Len()
    return dist,clamped

#Draw a single geodesic line from a specified point at a specified angle and ref0 A reference direction approximately along the axis of the part    
def createGeoLine(tbm, startPt, startFace, theta, ref0, calc_thick = False, tw = 6.35, maxPathLength = 1000, lim = (np.Inf,np.Inf, np.Inf), tol=1e-4):
    valid = True
    fail = 0
    length = 0
    ds = []
    startVec = rotAxisAngle(ref0,tbm.faces[startFace].normal,theta)
    startVN = P3.Cross(startVec,tbm.faces[startFace].normal)
    
    #find which bar intersects with the start vector
    for b in tbm.faces[startFace].bars:
        p1 = b.GetNodeFore(False).p
        p2 = b.GetNodeFore(True).p
        v = p2-p1
        if (P3.Dot((p1-startPt),startVN) > 0) is not (P3.Dot((p2-startPt),startVN) > 0):
            lam = (P3.Dot(startPt,startVN)-P3.Dot(p1,startVN))/(P3.Dot(p2,startVN)-P3.Dot(p1,startVN))
            nextpt = p1 + v*lam
            if P3.Dot((nextpt-startPt),startVec) > 0:
                bar = b
                bGoRight = bar.faceleft == startFace
                break
    
    pts = [startPt]
    faces = [startFace]
    curvs = [0]
    edges = [P3(0,0,0)]
    finished = False
    bar_last = tbm.bars[0]
    nodes = [] # list of nodes that fall within half a tow's width of the geoline
    A = startPt
    if type(calc_thick) == list:
        nodes = [False] * len(thickPts)
    while not finished:
        if bar != None:
            if bGoRight:
                faces.append(bar.faceright)
            else:
                faces.append(bar.faceleft)
            bar_last = bar
            if bGoRight:
                edges.append(bar.GetNodeFore(True).p - bar.GetNodeFore(False).p)
            else:
                edges.append(bar.GetNodeFore(False).p - bar.GetNodeFore(True).p)
            pt, bar, lam, bGoRight = GeoCrossBar(pts[-1], bar, lam, bGoRight)
            pts.append(pt)
            
            
            #Thickness calc
            if calc_thick and type(calc_thick) != list:
                B = pt
                check_bars = [bar_last]
                i=0
                while i < len(check_bars):
                    nei = []
                    df,c = distPointLine(A,B,check_bars[i].GetNodeFore(True).p)
                    if df < tw/2 and check_bars[i].GetNodeFore(True) not in nodes:
                        nodes.append(check_bars[i].GetNodeFore(True))
                    db,c = distPointLine(A,B,check_bars[i].GetNodeFore(False).p)
                    if db < tw/2 and check_bars[i].GetNodeFore(False) not in nodes:
                        nodes.append(check_bars[i].GetNodeFore(False))

                    if check_bars[i].GetForeRightBL(True) and df < tw/2:
                        nei.append(check_bars[i].GetForeRightBL(True))
                    if check_bars[i].GetForeRightBL(False) and db < tw/2:
                        nei.append(check_bars[i].GetForeRightBL(False))
                    for b in nei:
                        if b not in check_bars:
                            check_bars.append(b)
                    i += 1
                A = B
        
        d = P3.Len(pt-pts[-2])
        length += d
        ds.append(d)
        if bar==None:
            finished =True
            print('edge reached on path', theta, 'deg')
            if bar_last.badedge:
                print('PATH FAIL: bad edge reached')
                valid = False
                fail = 1
        elif length>maxPathLength:
            finished =True
            print('PATH FAIL: max path length reached on path', theta)
            valid = False
            fail = 2
        elif pt.x > lim[0]:
            finished = True
        
        elif pt.y > lim[1] and len(pts)>1:
            l = (lim[1]-pts[-2].y) / (pt.y-pts[-2].y)
            pts[-1] = pts[-2] + (pt-pts[-2])*l
            faces[-1] = faces[-2]
            finished = True
        elif pt.z > lim[2]:
            finished = True
        else:
            finished =False
            
    curvs = [0]
    for i in range(1,len(pts)-1):
        vlast = pts[i] - pts[i-1]
        vnext = pts[i+1] - pts[i]
        curvature = P3.Dot(P3.ZNorm(tbm.faces[faces[i]].normal),P3.ZNorm(vlast))    
        curvs.append(curvature)
    curvs.append(0)

    smcurvs = []
    av = 100 #Number of points forwards and backwards to use in curvature calculation
    for i in range(len(pts)):
        lo = max(0,i-av)
        hi = min(i+av+1,len(pts))
        sm = 0
        for j in range(lo,hi):
            d = 1+(sum(ds[j:i])+sum(ds[i:j]))/tbm.meshsize
            c = curvs[j]
            sm += (c/d**2)/tbm.meshsize
        smcurvs.append(sm)
    if min(smcurvs) < 0:
        print('PATH FAIL: concave area of',min(smcurvs),'on path', theta)
        valid = False
        fail = 3
        
    return {'pts':pts,'curvs':smcurvs,'faces':faces, 'nodes':nodes, 'edges':edges, 'valid':valid, 'fail':fail,'length':length}
